In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sympy import *
from robot_tools import *
import math

In [2]:
def velocity_COM(frame, transform_low_high, P, v, omega, alias):
    rotation_high_low = transform_low_high.T[:3, :3]

    symprint('\Omega_G', frame, frame)
    OmegaGee = simplify(rotation_high_low * omega)
    matprint(OmegaGee, alias)

    symprint('V_G', frame, frame)
    VGee = simplify(v + omega.cross(P))
    matprint(VGee, alias)
    
    return OmegaGee, VGee

### 5.7.1 Variables

In [3]:
t = symbols('t')
theta1 = Function(r"\theta_1")(t)
theta1_dot = diff(theta1, t)
theta1_ddot = diff(theta1, t, t)
lg = symbols('l_g')
l = symbols('l')

alias = {}

# display as theta dot instead of d(theta)/dt, and ignore dependency (t)
alias.update({theta1: symbols(r"\theta_1"),
              theta1_dot: symbols(r"\dot{\theta_1}"),
              theta1_ddot: symbols(r"\ddot{\theta_1}"),
             })

In [4]:
# Q1

DH1 = [0, 0, 0, theta1]
T01 = Tlink(DH1)
symprint('T',0, 1)
matprint(T01, alias)

# IMPORTANT: Treat the centre of mass as a separate joint!
DH2 = [lg, 0, 0, 0]
T1G = Tlink(DH2)
symprint('T',1, 'G')
matprint(T1G, alias)

T0G = simplify(T01 * T1G)
symprint('T', 0, 'G')
matprint(T0G, alias)

DHE = [l, 0, 0, 0]
T1E = Tlink(DHE)
symprint('T',1, 'E')
matprint(T1E, alias)

T0E = simplify(T01 * T1E)
symprint('T', 0, 'E')
matprint(T0E, alias)


^0T_1

Matrix([
[cos(\theta_1), -sin(\theta_1), 0, 0],
[sin(\theta_1),  cos(\theta_1), 0, 0],
[            0,              0, 1, 0],
[            0,              0, 0, 1]])

^1T_G

Matrix([
[1, 0, 0, l_g],
[0, 1, 0,   0],
[0, 0, 1,   0],
[0, 0, 0,   1]])

^0T_G

Matrix([
[cos(\theta_1), -sin(\theta_1), 0, l_g*cos(\theta_1)],
[sin(\theta_1),  cos(\theta_1), 0, l_g*sin(\theta_1)],
[            0,              0, 1,                 0],
[            0,              0, 0,                 1]])

^1T_E

Matrix([
[1, 0, 0, l],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

^0T_E

Matrix([
[cos(\theta_1), -sin(\theta_1), 0, l*cos(\theta_1)],
[sin(\theta_1),  cos(\theta_1), 0, l*sin(\theta_1)],
[            0,              0, 1,               0],
[            0,              0, 0,               1]])

In [5]:
# Q2

omega = Matrix([0, 0, 0])
v = Matrix([0, 0, 0])

omega_11, v_11 = revolute_joint(frame = 1, 
                                theta_dot = theta1_dot, 
                                transform_low_high = T01, 
                                omega = omega, v = v, 
                                Display = False,
                                alias = alias)


# IMPORTANT: Treat the centre of mass as a separate joint!
omega_GG, v_GG = revolute_joint(frame = 'e', 
                                theta_dot = theta1_dot, 
                                transform_low_high = T1G, 
                                omega = omega_11, v = v_11, 
                                alias = alias)


Jee, J0 = Jacobian(parameters = [theta1_dot], 
                     v_ee = v_GG, omega_ee = omega_GG, 
                     transform_low_high = T0G, 
                     alias = alias, Display=True, Display_all_details=False)

# Find velocity of G in frame {0}
symprint('V', 0, 'G')
v_0G = simplify(T0G[:3, :3] * v_GG)
matprint(v_0G, alias)

symprint('\\Omega', 0, 'G')
omega_0G = simplify(T0G[:3, :3] * omega_GG)
matprint(omega_0G, alias)


^e\Omega_e

Matrix([
[             0],
[             0],
[\dot{\theta_1}]])

^eV_e

Matrix([
[                 0],
[\dot{\theta_1}*l_g],
[                 0]])

^eJ_e

...

Matrix([
[-l_g*sin(\theta_1)],
[ l_g*cos(\theta_1)],
[                 0],
[                 0],
[                 0],
[                 1]])

^0V_G

Matrix([
[-\dot{\theta_1}*l_g*sin(\theta_1)],
[ \dot{\theta_1}*l_g*cos(\theta_1)],
[                                0]])

^0\Omega_G

Matrix([
[             0],
[             0],
[\dot{\theta_1}]])

In [6]:
# Q3

# Differentiate with respect to t

symprint('\dot{V}', 0, 'G')

A0G = simplify(v_0G.diff(t))
matprint(A0G, alias)

^0\dot{V}_G

Matrix([
[-l_g*(\ddot{\theta_1}*sin(\theta_1) + \dot{\theta_1}**2*cos(\theta_1))],
[ l_g*(\ddot{\theta_1}*cos(\theta_1) - \dot{\theta_1}**2*sin(\theta_1))],
[                                                                     0]])

In [7]:
# Q4

# Find velocity of G in frame {1}

symprint('V', 1, 'G')
v_1G = simplify(T1G[:3, :3] * v_GG)
matprint(v_1G, alias)

symprint('\\Omega', 0, 'G')
omega_1G = simplify(T1G[:3, :3] * omega_GG)
matprint(omega_1G, alias)

#------------------------------

# Find acceleration

# DON'T DIRECTLY APPLY ELEMENT-WISE DIFFERENTIATE, EXCEPT WITH RESPECT TO {0}
# Differentiate with respect to t
# symprint('\dot{V}', 1, 'G')
# A1G = simplify(v_1G.diff(t))
# matprint(A1G, alias)

# Find A1G by transforming from A0G
symprint('\dot{V}', 1, 'G')
A1G = simplify(T01[:3, :3]**-1 * A0G)
matprint(A1G, alias)

^1V_G

Matrix([
[                 0],
[\dot{\theta_1}*l_g],
[                 0]])

^0\Omega_G

Matrix([
[             0],
[             0],
[\dot{\theta_1}]])

^1\dot{V}_G

Matrix([
[-\dot{\theta_1}**2*l_g],
[   \ddot{\theta_1}*l_g],
[                     0]])

In [8]:
# Another method to compute v_1G. See Chapter 4 Equation 13
matprint(simplify(v_11 + omega_11.cross(Matrix([lg, 0, 0]))), alias)

Matrix([
[                 0],
[\dot{\theta_1}*l_g],
[                 0]])

In [9]:
# Q5

def angular_acce(frame, joint_type, transform_low_high, omega_prev, omega_dot_prev, theta_dot, theta_ddot, alias, Display = True):
    
    '''
    See Chapter 5 Equation 32 and 33 for details
    Note on the indices:
    omega_prev: i
    omega_dot_prev: i
    theta_dot: i+1
    theta_ddot: i+1
    output: i+1
    '''
    
    # Transpose and extract the 3x3 matrix
    rotation_high_low = transform_low_high.T[:3, :3]
    
    if joint_type == 'r':
        # Equation 32
        omega_new = simplify(rotation_high_low * omega_dot_prev +
                             rotation_high_low * omega_prev.cross(theta_dot * Matrix([0, 0, 1])) +
                             theta_ddot * Matrix([0, 0, 1])
                            )
        if Display == True:
            if type(frame) == int:
                symprint('\dot{\\Omega}', frame, frame)
            else:
                symprint('\dot{\\Omega}', '', frame)  
            matprint(omega_new, alias)
            
    elif joint_type == 'p':
        # Equation 33
        omega_new = simplify(rotation_high_low * omega_dot_prev)
        
        if Display == True:
            symprint('\dot{\\Omega}', frame, frame)
            matprint(omega_new, alias)
            
    else:
        print('Joint type not found, try r and p')
        return None
    
    
    return omega_new


def linear_acce(frame, joint_type, transform_low_high, omega_prev, omega_dot_prev, v_dot_prev, alias, Display = True):
    
    '''
    See Chapter 5 Equation 34 and 35 for details
    Note on the indices:
    omega_prev: i
    omega_dot_prev: i
    v_dot_prev: i
    output: i+1
    '''
    
    
    # Transpose and extract the 3x3 matrix
    rotation_high_low = transform_low_high.T[:3, :3]
    P = transform_low_high[:3, -1]
    
    if joint_type == 'r':
        
        # Equation 34
        v_new = simplify(rotation_high_low * (v_dot_prev + omega_dot_prev.cross(P) + omega_prev.cross(omega_prev.cross(P))))
        
        if Display == True:
            if type(frame) == int:
                symprint('\dot{V}', frame, frame)
            else:
                symprint('\dot{V}', '', frame)
            matprint(v_new, alias)
    elif joint_type == 'p':
        
        # Equation 35 (did not implement)
        pass
        
    else:
        print('Joint type not found, try r and p')
        return None
    
    return v_new

def force_cal(mass, v_dot):
    F = simplify(mass * v_dot)  
    return F
    
def moment_cal(I, omega_dot, omega):
    
    intermediate = I * omega
    M = simplify(I * omega_dot + omega.cross(intermediate))
    return M

    
    

In [10]:
# Outward iteration to find the Force and moment on the COM
omega_00 = Matrix([0, 0, 0])
omega_dot_00 = Matrix([0, 0, 0])
v_00 = Matrix([0, 0, 0])
g = symbols('g')
v_dot_00 = Matrix([0, 0, g])


omega_11, v_11 = revolute_joint(frame = 1, 
                                theta_dot = theta1_dot, 
                                transform_low_high = T01, 
                                omega = omega, v = v, 
                                Display = False,
                                alias = alias)

omega_dot_11 = angular_acce(frame = 1,
                 joint_type = 'r',
                 transform_low_high = T01,
                 omega_prev = omega_00,
                 omega_dot_prev = omega_dot_00,
                 theta_dot = theta1_dot,
                 theta_ddot = theta1_ddot,
                 alias = alias)

v_dot_11 = linear_acce(frame = 1,
                 joint_type = 'r',
                 transform_low_high = T01,
                 omega_prev = omega_00,
                 omega_dot_prev = omega_dot_00,
                 v_dot_prev = v_dot_00,
                 alias = alias)

v_dot_1G = linear_acce(frame = 'G',
                 joint_type = 'r',
                 transform_low_high = T1G,
                 omega_prev = omega_11,
                 omega_dot_prev = omega_dot_11,
                 v_dot_prev = v_dot_11,
                 alias = alias)

# Equation 36
V_dot_1G = simplify(v_dot_11 + omega_dot_11.cross(Matrix([lg, 0, 0])) + omega_11.cross(omega_11.cross(Matrix([lg, 0, 0]))))
matprint(V_dot_1G, alias)

# Calculate force
Ixx = symbols('I_{xx}')
Iyy = symbols('I_{yy}')
Izz = symbols('I_{zz}')

I = Matrix([[Ixx, 0, 0],
            [0, Iyy, 0],
            [0, 0, Izz]])

m1 = symbols('m_1')

F1G = force_cal(mass = m1, 
              v_dot = v_dot_1G)
symprint('F', 1, 'G')
matprint(F1G, alias)

M1G = moment_cal(I = I, 
               omega_dot = omega_dot_11, 
               omega = omega_11)
symprint('M', 1, 'G')
matprint(M1G, alias)


^1\dot{\Omega}_1

Matrix([
[              0],
[              0],
[\ddot{\theta_1}]])

^1\dot{V}_1

Matrix([
[0],
[0],
[g]])

\dot{V}_G

...

Matrix([
[-\dot{\theta_1}**2*l_g],
[   \ddot{\theta_1}*l_g],
[                     g]])

^1F_G

Matrix([
[-\dot{\theta_1}**2*l_g*m_1],
[   \ddot{\theta_1}*l_g*m_1],
[                     g*m_1]])

^1M_G

Matrix([
[                     0],
[                     0],
[I_{zz}*\ddot{\theta_1}]])

In [11]:
def inward_iteration(transform_low_high_com, transform_low_high, Fnext, Fcom, Mnext, Mcom):
 
    rotation_low_high = transform_low_high[:3, :3]
    
    # Position of the center of mass
    Pcom = transform_low_high_com[:3, -1]
    
    # Position of the next joint
    P = transform_low_high[:3, -1]
    
    Fprev = simplify(rotation_low_high * Fnext + Fcom)
    Mprev = simplify(Mcom + rotation_low_high * Mnext + Pcom.cross(Fcom) + P.cross(rotation_low_high * Fnext))
    
    return Fprev, Mprev



In [12]:
Fee = Matrix([0, 0, 0])
Mee = Matrix([0, 0, 0])
F11, M11 = inward_iteration(transform_low_high_com = T1G, 
                       transform_low_high = T1E,
                       Fnext = Fee, 
                       Fcom = F1G,
                       Mnext = Mee,
                       Mcom = M1G)

matprint(F11, alias)
matprint(M11, alias)

Matrix([
[-\dot{\theta_1}**2*l_g*m_1],
[   \ddot{\theta_1}*l_g*m_1],
[                     g*m_1]])

Matrix([
[                                    0],
[                           -g*l_g*m_1],
[\ddot{\theta_1}*(I_{zz} + l_g**2*m_1)]])

### 5.7.2 Variables

In [13]:
t = symbols('t')
theta1 = Function(r"\theta_1")(t)
theta1_dot = diff(theta1, t)
theta1_ddot = diff(theta1, t, t)

theta2 = Function(r"\theta_2")(t)
theta2_dot = diff(theta2, t)
theta2_ddot = diff(theta2, t, t)

L1 = symbols('L1')
L2 = symbols('L2')
Lg1 = symbols('L_{g1}')
Lg2 = symbols('L_{g2}')

alias = {}

# display as theta dot instead of d(theta)/dt, and ignore dependency (t)
alias.update({theta1: symbols(r"\theta_1"),
              theta1_dot: symbols(r"\dot{\theta_1}"),
              theta1_ddot: symbols(r"\ddot{\theta_1}"),
              theta2: symbols(r"\theta_2"),
              theta2_dot: symbols(r"\dot{\theta_2}"),
              theta2_ddot: symbols(r"\ddot{\theta_2}")
             })

In [14]:
# Q1

DH1 = [0, 0, 0, theta1]
T01 = Tlink(DH1)
symprint('T',0, 1)
matprint(T01)

DH2 = [L1, -pi/2, 0, theta2]
T12 = Tlink(DH2)
symprint('T',1, 2)
matprint(T12)

T02 = simplify(T01 * T12)
symprint('T',0, 2)
matprint(T02)

# -------------------

# Treat COM as another joint

DH1G = [Lg1, 0, 0, 0]
T1G1 = Tlink(DH1G)
symprint('T',1, 'g1')
matprint(T1G1)

DH2G = [Lg2, 0, 0, 0]
T2G2 = Tlink(DH2G)
symprint('T', 2, 'g2')
matprint(T2G2)

T0G2 = simplify(T01 * T12 * T2G2)
symprint('T', 0, 'G2')
matprint(T0G2)

^0T_1

Matrix([
[cos(\theta_1(t)), -sin(\theta_1(t)), 0, 0],
[sin(\theta_1(t)),  cos(\theta_1(t)), 0, 0],
[               0,                 0, 1, 0],
[               0,                 0, 0, 1]])

^1T_2

Matrix([
[ cos(\theta_2(t)), -sin(\theta_2(t)), 0, L1],
[                0,                 0, 1,  0],
[-sin(\theta_2(t)), -cos(\theta_2(t)), 0,  0],
[                0,                 0, 0,  1]])

^0T_2

...

Matrix([
[1, 0, 0, L_{g1}],
[0, 1, 0,      0],
[0, 0, 1,      0],
[0, 0, 0,      1]])

^2T_g2

Matrix([
[1, 0, 0, L_{g2}],
[0, 1, 0,      0],
[0, 0, 1,      0],
[0, 0, 0,      1]])

^0T_G2

Matrix([
[cos(\theta_1(t))*cos(\theta_2(t)), -sin(\theta_2(t))*cos(\theta_1(t)), -sin(\theta_1(t)), (L1 + L_{g2}*cos(\theta_2(t)))*cos(\theta_1(t))],
[sin(\theta_1(t))*cos(\theta_2(t)), -sin(\theta_1(t))*sin(\theta_2(t)),  cos(\theta_1(t)), (L1 + L_{g2}*cos(\theta_2(t)))*sin(\theta_1(t))],
[                -sin(\theta_2(t)),                  -cos(\theta_2(t)),                 0,                        -L_{g2}*sin(\theta_2(t))],
[                                0,                                  0,                 0,                                               1]])

In [15]:
# Q2

omega = Matrix([0, 0, 0])
v = Matrix([0, 0, 0])

omega_11, v_11 = revolute_joint(frame = 1, 
                                theta_dot = theta1_dot, 
                                transform_low_high = T01, 
                                omega = omega, v = v, 
                                Display = False,
                                alias = alias)

omega_22, v_22 = revolute_joint(frame = 2, 
                                theta_dot = theta2_dot, 
                                transform_low_high = T12, 
                                omega = omega_11, v = v_11, 
                                Display = False,
                                alias = alias)

omega_G2G2, v_G2G2 = revolute_joint(frame = 'e', 
                                theta_dot = None, 
                                transform_low_high = T2G2, 
                                omega = omega_22, v = v_22, 
                                Display = False,
                                alias = alias)

# V_2G2 = R2G * V_G2G2
v_2G2 = simplify(T2G2[:3, :3] * v_G2G2)

# Omega_2G2 = R2G * Omega_G2G2
omega_2G2 = simplify(T2G2[:3, :3] * omega_G2G2)

symprint('V', 2, 'g2')
matprint(v_2G2, alias)

symprint('\\Omega', 2, 'g2')
matprint(omega_2G2, alias)


^2V_g2

Matrix([
[                                         0],
[                     L_{g2}*\dot{\theta_2}],
[\dot{\theta_1}*(L1 + L_{g2}*cos(\theta_2))]])

^2\Omega_g2

Matrix([
[-\dot{\theta_1}*sin(\theta_2)],
[-\dot{\theta_1}*cos(\theta_2)],
[               \dot{\theta_2}]])

In [16]:
# Another method to compute v_2G2. See Chapter 4 Equation 13
symprint('V', 2, 'g2')
matprint(simplify(v_22 + omega_22.cross(Matrix([Lg2, 0, 0]))), alias)

^2V_g2

Matrix([
[                                         0],
[                     L_{g2}*\dot{\theta_2}],
[\dot{\theta_1}*(L1 + L_{g2}*cos(\theta_2))]])

In [17]:
# Q3

v_0G2 = simplify(T0G2[:3, :3] * v_G2G2)

symprint('V', 0, '{g2}')
matprint(v_0G2, alias)

^0V_{g2}

Matrix([
[-L_{g2}*\dot{\theta_2}*sin(\theta_2)*cos(\theta_1) - \dot{\theta_1}*(L1 + L_{g2}*cos(\theta_2))*sin(\theta_1)],
[-L_{g2}*\dot{\theta_2}*sin(\theta_1)*sin(\theta_2) + \dot{\theta_1}*(L1 + L_{g2}*cos(\theta_2))*cos(\theta_1)],
[                                                                         -L_{g2}*\dot{\theta_2}*cos(\theta_2)]])

In [18]:
# This answer is checked and it is correct
v_dot_0G2 = simplify(v_0G2.diff(t))

symprint('\dot{V}', 0, '{g2}')
matprint(v_dot_0G2, alias)

# This answer is not checked
v_dot_2G2 = simplify(T02[:3, :3].T * v_dot_0G2)

symprint('\dot{V}', 2, '{g2}')
matprint(v_dot_2G2, alias)

^0\dot{V}_{g2}

Matrix([
[-L_{g2}*\ddot{\theta_2}*sin(\theta_2)*cos(\theta_1) + 2*L_{g2}*\dot{\theta_1}*\dot{\theta_2}*sin(\theta_1)*sin(\theta_2) - L_{g2}*\dot{\theta_2}**2*cos(\theta_1)*cos(\theta_2) - \ddot{\theta_1}*(L1 + L_{g2}*cos(\theta_2))*sin(\theta_1) - \dot{\theta_1}**2*(L1 + L_{g2}*cos(\theta_2))*cos(\theta_1)],
[-L_{g2}*\ddot{\theta_2}*sin(\theta_1)*sin(\theta_2) - 2*L_{g2}*\dot{\theta_1}*\dot{\theta_2}*sin(\theta_2)*cos(\theta_1) - L_{g2}*\dot{\theta_2}**2*sin(\theta_1)*cos(\theta_2) + \ddot{\theta_1}*(L1 + L_{g2}*cos(\theta_2))*cos(\theta_1) - \dot{\theta_1}**2*(L1 + L_{g2}*cos(\theta_2))*sin(\theta_1)],
[                                                                                                                                                                                                                                L_{g2}*(-\ddot{\theta_2}*cos(\theta_2) + \dot{\theta_2}**2*sin(\theta_2))]])

^2\dot{V}_{g2}

Matrix([
[      -L1*\dot{\theta_1}**2*cos(\theta_2) - L_{g2}*\dot{\theta_1}**2*cos(\theta_2)**2 - L_{g2}*\dot{\theta_2}**2],
[        L1*\dot{\theta_1}**2*sin(\theta_2) + L_{g2}*\ddot{\theta_2} + L_{g2}*\dot{\theta_1}**2*sin(2*\theta_2)/2],
[L1*\ddot{\theta_1} + L_{g2}*\ddot{\theta_1}*cos(\theta_2) - 2*L_{g2}*\dot{\theta_1}*\dot{\theta_2}*sin(\theta_2)]])

In [19]:
# Q4
# Calculate force
Ixx = symbols('I_{xx}')
Iyy = symbols('I_{yy}')
Izz = symbols('I_{zz}')

I = Matrix([[Ixx, 0, 0],
            [0, Iyy, 0],
            [0, 0, Izz]])

m2 = symbols('m_2')

F2G2 = force_cal(mass = m2, v_dot = v_dot_2G2)
symprint('F', 2, 'G2')
matprint(F2G2, alias)

M2G2 = moment_cal(I = I, 
               omega_dot = omega_dot_11, 
               omega = omega_11)
symprint('M', 1, 'G')
matprint(M1G, alias)

^2F_G2

Matrix([
[      -m_2*(L1*\dot{\theta_1}**2*cos(\theta_2) + L_{g2}*\dot{\theta_1}**2*cos(\theta_2)**2 + L_{g2}*\dot{\theta_2}**2)],
[    m_2*(2*L1*\dot{\theta_1}**2*sin(\theta_2) + 2*L_{g2}*\ddot{\theta_2} + L_{g2}*\dot{\theta_1}**2*sin(2*\theta_2))/2],
[m_2*(L1*\ddot{\theta_1} + L_{g2}*\ddot{\theta_1}*cos(\theta_2) - 2*L_{g2}*\dot{\theta_1}*\dot{\theta_2}*sin(\theta_2))]])

^1M_G

Matrix([
[                     0],
[                     0],
[I_{zz}*\ddot{\theta_1}]])

In [22]:
# Q4

# Outward iteration to find the Force and moment on the COM
omega_00 = Matrix([0, 0, 0])
omega_dot_00 = Matrix([0, 0, 0])
v_00 = Matrix([0, 0, 0])
g = symbols('g')
v_dot_00 = Matrix([0, g, 0])


omega_11, v_11 = revolute_joint(frame = 1, 
                                theta_dot = theta1_dot, 
                                transform_low_high = T01, 
                                omega = omega, v = v, 
                                Display = False,
                                alias = alias)

omega_dot_11 = angular_acce(frame = 1,
                 joint_type = 'r',
                 transform_low_high = T01,
                 omega_prev = omega_00,
                 omega_dot_prev = omega_dot_00,
                 theta_dot = theta1_dot,
                 theta_ddot = theta1_ddot,
                 alias = alias)

v_dot_11 = linear_acce(frame = 1,
                 joint_type = 'r',
                 transform_low_high = T01,
                 omega_prev = omega_00,
                 omega_dot_prev = omega_dot_00,
                 v_dot_prev = v_dot_00,
                 alias = alias)

omega_22, v_22 = revolute_joint(frame = 2, 
                                theta_dot = theta2_dot, 
                                transform_low_high = T12, 
                                omega = omega_11, v = v_11, 
                                Display = False,
                                alias = alias)

omega_dot_22 = angular_acce(frame = 2,
                 joint_type = 'r',
                 transform_low_high = T12,
                 omega_prev = omega_11,
                 omega_dot_prev = omega_dot_11,
                 theta_dot = theta2_dot,
                 theta_ddot = theta2_ddot,
                 alias = alias)

v_dot_22 = linear_acce(frame = 2,
                 joint_type = 'r',
                 transform_low_high = T12,
                 omega_prev = omega_11,
                 omega_dot_prev = omega_dot_11,
                 v_dot_prev = v_dot_11,
                 alias = alias)

V_dot_2G2 = simplify(v_dot_22 + omega_dot_22.cross(Matrix([Lg2, 0, 0])) + omega_22.cross(omega_22.cross(Matrix([Lg2, 0, 0]))))


symprint('\dot{V}', 2, '{g2}')
matprint(V_dot_2G2, alias)

^1\dot{\Omega}_1

Matrix([
[              0],
[              0],
[\ddot{\theta_1}]])

^1\dot{V}_1

Matrix([
[g*sin(\theta_1)],
[g*cos(\theta_1)],
[              0]])

^2\dot{\Omega}_2

Matrix([
[-\ddot{\theta_1}*sin(\theta_2)],
[-\ddot{\theta_1}*cos(\theta_2)],
[               \ddot{\theta_2}]])

^2\dot{V}_2

Matrix([
[(-L1*\dot{\theta_1}**2 + g*sin(\theta_1))*cos(\theta_2)],
[ (L1*\dot{\theta_1}**2 - g*sin(\theta_1))*sin(\theta_2)],
[                   L1*\ddot{\theta_1} + g*cos(\theta_1)]])

^2\dot{V}_{g2}

Matrix([
[  -L_{g2}*\dot{\theta_1}**2*cos(\theta_2)**2 - L_{g2}*\dot{\theta_2}**2 - (L1*\dot{\theta_1}**2 - g*sin(\theta_1))*cos(\theta_2)],
[    L_{g2}*\ddot{\theta_2} + L_{g2}*\dot{\theta_1}**2*sin(2*\theta_2)/2 + (L1*\dot{\theta_1}**2 - g*sin(\theta_1))*sin(\theta_2)],
[L1*\ddot{\theta_1} + L_{g2}*\ddot{\theta_1}*cos(\theta_2) - L_{g2}*\dot{\theta_1}*\dot{\theta_2}*sin(\theta_2) + g*cos(\theta_1)]])

In [21]:
# This answer is checked and it is correct
v_dot_0G2 = simplify(v_0G2.diff(t))

symprint('\dot{V}', 0, '{g2}')
matprint(v_dot_0G2, alias)

# This answer is not checked
v_dot_2G2 = simplify(T02[:3, :3].T * v_dot_0G2)

symprint('\dot{V}', 2, '{g2}')
matprint(v_dot_2G2, alias)

^0\dot{V}_{g2}

Matrix([
[-L_{g2}*\ddot{\theta_2}*sin(\theta_2)*cos(\theta_1) + 2*L_{g2}*\dot{\theta_1}*\dot{\theta_2}*sin(\theta_1)*sin(\theta_2) - L_{g2}*\dot{\theta_2}**2*cos(\theta_1)*cos(\theta_2) - \ddot{\theta_1}*(L1 + L_{g2}*cos(\theta_2))*sin(\theta_1) - \dot{\theta_1}**2*(L1 + L_{g2}*cos(\theta_2))*cos(\theta_1)],
[-L_{g2}*\ddot{\theta_2}*sin(\theta_1)*sin(\theta_2) - 2*L_{g2}*\dot{\theta_1}*\dot{\theta_2}*sin(\theta_2)*cos(\theta_1) - L_{g2}*\dot{\theta_2}**2*sin(\theta_1)*cos(\theta_2) + \ddot{\theta_1}*(L1 + L_{g2}*cos(\theta_2))*cos(\theta_1) - \dot{\theta_1}**2*(L1 + L_{g2}*cos(\theta_2))*sin(\theta_1)],
[                                                                                                                                                                                                                                L_{g2}*(-\ddot{\theta_2}*cos(\theta_2) + \dot{\theta_2}**2*sin(\theta_2))]])

^2\dot{V}_{g2}

Matrix([
[      -L1*\dot{\theta_1}**2*cos(\theta_2) - L_{g2}*\dot{\theta_1}**2*cos(\theta_2)**2 - L_{g2}*\dot{\theta_2}**2],
[        L1*\dot{\theta_1}**2*sin(\theta_2) + L_{g2}*\ddot{\theta_2} + L_{g2}*\dot{\theta_1}**2*sin(2*\theta_2)/2],
[L1*\ddot{\theta_1} + L_{g2}*\ddot{\theta_1}*cos(\theta_2) - 2*L_{g2}*\dot{\theta_1}*\dot{\theta_2}*sin(\theta_2)]])